In [242]:
import pandas as pd
import numpy as np
import datetime as dt
import re
import sys

In [219]:
senate_data = pd.read_csv('/Users/Brandon/indEng_data_x/tenguins/data/cleaned/RCP_senate.csv')
senate_DemRec = pd.read_csv('/Users/Brandon/indEng_data_x/tenguins/data/cleaned/RCP_senate_DemRec.csv')
p_approval = pd.read_csv('/Users/Brandon/indEng_data_x/tenguins/data/cleaned/RCP_p_approval.csv')
purified_polls = pd.read_csv('/Users/Brandon/indEng_data_x/tenguins/data/cleaned/purified_polls.csv')
c_approval = pd.read_csv('/Users/Brandon/indEng_data_x/tenguins/data/cleaned/RCP_c_approval.csv')
# approval = pd.read_csv('/Users/Brandon/indEng_data_x/tenguins/data/cleaned/RCP_approval.csv')
polls_copy = purified_polls


In [255]:
####################
# helper functions #
####################


flatten = lambda lst: [item for sublist in lst for item in sublist]  


def containsAny(str, set):
    return 1 in [element in str for element in set]


def quantizeSpread(lst):
    disapprovalSet = {'disapprove', 'Disapprove', 'Wrong Direction', 'Wrong Track'}
    tieSet = {'Tie', 'tie'}
    for i in range(len(lst)):
        if containsAny(lst[i], disapprovalSet):
            lst[i] = re.sub(r'[^0-9]+', "-", lst[i])
        elif containsAny(lst[i], tieSet):
            lst[i] = re.sub(r'[^0-9]+', "0", lst[i])
        else:
            lst[i] = re.sub(r'[^0-9]', "", lst[i])     
    if flatten([re.findall(r'[^0-9\-]', i) for i in lst]):
        raise Exception(lst)
    return lst

def approvalTableCleanup(table):
    splitColumn = [i.split(',') for i in table['Results']]
    approvalCol = [splitColumn[i][0] for i in range(len(splitColumn))]
    disapprovalCol = [splitColumn[i][1] for i in range(len(splitColumn))]
    table['approval rating'] = [re.sub(r'[^0-9]', "", i) for i in approvalCol]
    table['disapproval rating'] = [re.sub(r'[^0-9]', "", i) for i in disapprovalCol]
    table['year'] = pd.DatetimeIndex(table['Date']).year
    table['month'] = pd.DatetimeIndex(table['Date']).month
    table = table.drop('Results', axis = 1)
    stringSpreads = table['Spread'].tolist()
    table['spread'] = quantizeSpread(stringSpreads)
    table = table.drop('Spread', axis = 1)
    table = table.drop('Unnamed: 0', axis = 1)
    table.columns = map(str.lower, table.columns)
    table = table.rename(index = str, columns = {"race/topic (click to sort)": "approval type"})
    table['approval type'] = table['approval type'].str.lower()
    table['poll'] = table['poll'].str.lower()
    return table

updated_p_approval = approvalTableCleanup(p_approval)
updated_c_approval = approvalTableCleanup(c_approval)

In [230]:
updated_p_approval.to_csv('updated_p_approval')
updated_c_approval.to_csv('updated_c_approval')

In [228]:
# new_approval = p_approval
# full_results = new_approval['Results'].tolist()
# splitVals = [i.split(',') for i in new_approval['Results']]
# approvalCol = [splitVals[i][0] for i in range(len(splitVals))]
# disapprovalCol = [splitVals[i][1] for i in range(len(splitVals))]

# approvalRatings = [re.sub(r'[^0-9]', "", i) for i in approvalCol]




In [253]:
test = ['1','-2']


if flatten([re.findall(r'[^0-9\-]', i) for i in test]):
    print('b')

In [85]:
senate_data.head()

,Unnamed: 0,Race/Topic (Click to Sort),Poll,Results,Spread,Date,Year
0,0,Florida Senate - Rubio vs. Meek vs. Crist,Quinnipiac,"Crist 32, Rubio 30, Meek 24",Crist +2,2010-04-15,2010.0
1,1,Arkansas Senate - Boozman vs. Lincoln,R2000/Daily Kos (D),"Boozman 50, Lincoln 43",Boozman +7,2010-04-15,2010.0
2,2,Arkansas Senate - Boozman vs. Halter,R2000/Daily Kos (D),"Boozman 48, Halter 41",Boozman +7,2010-04-15,2010.0
3,3,Florida Senate - Rubio vs. Meek,Quinnipiac,"Rubio 42, Meek 38",Rubio +4,2010-04-15,2010.0
4,4,Florida Senate - Republican Primary,Quinnipiac,"Rubio 56, Crist 33",Rubio +23,2010-04-15,2010.0


In [6]:
senate_DemRec.head()

,Date,State,Race,Poll,Result,Spread,Year,Lead,Party,Party Class
0,2012-04-19,Missouri,Missouri Senate - Akin vs. McCaskill,Rasmussen Reports,"Akin 48, McCaskill 43",Akin +5,2012,Akin,R,0.0
1,2012-07-28,Missouri,Missouri Senate - Akin vs. McCaskill,Post-Dispatch/Mason-Dixon,"McCaskill 44, Akin 49",Akin +5,2012,Akin,R,0.0
2,2012-08-01,Missouri,Missouri Senate - Akin vs. McCaskill,Rasmussen Reports,"McCaskill 44, Akin 47",Akin +3,2012,Akin,R,0.0
3,2012-08-13,Missouri,Missouri Senate - Akin vs. McCaskill,SurveyUSA*,"McCaskill 40, Akin 51",Akin +11,2012,Akin,R,0.0
4,2012-08-21,Missouri,Missouri Senate - Akin vs. McCaskill,PPP (D),"McCaskill 43, Akin 44",Akin +1,2012,Akin,R,0.0


In [13]:
purified_polls.head()

,Unnamed: 0,Date,Poll,Race/Topic (Click to Sort),Results,Spread,Year
0,18998,2010-04-15,Quinnipiac,Florida Senate - Rubio vs. Meek vs. Crist,"Crist 32, Rubio 30, Meek 24",Crist +2,2010
1,18992,2010-04-15,R2000/Daily Kos (D),Arkansas Senate - Boozman vs. Lincoln,"Boozman 50, Lincoln 43",Boozman +7,2010
2,18993,2010-04-15,R2000/Daily Kos (D),Arkansas Senate - Boozman vs. Halter,"Boozman 48, Halter 41",Boozman +7,2010
3,18997,2010-04-15,Quinnipiac,Florida Senate - Rubio vs. Meek,"Rubio 42, Meek 38",Rubio +4,2010
4,18995,2010-04-15,Quinnipiac,Florida Senate - Republican Primary,"Rubio 56, Crist 33",Rubio +23,2010


In [82]:
polls_copy['year'] = pd.DatetimeIndex(polls_copy['Date']).year
candidates_2018 = polls_copy[polls_copy['year'] == 2018]
candidates_2018.head()

,Unnamed: 0,Date,Poll,Race/Topic (Click to Sort),Results,Spread,Year,year
17400,1598,2018-01-01,Rasmussen Reports,Direction of Country,"Right Direction 40, Wrong Track 55",Wrong Track +15,2018,2018
17401,1591,2018-01-03,Economist/YouGov,President Trump Job Approval,"Approve 43, Disapprove 54",Disapprove +11,2018,2018
17402,1590,2018-01-03,Reuters/Ipsos,President Trump Job Approval,"Approve 41, Disapprove 54",Disapprove +13,2018,2018
17403,1592,2018-01-03,Economist/YouGov,Congressional Job Approval,"Approve 15, Disapprove 64",Disapprove +49,2018,2018
17404,1593,2018-01-03,Reuters/Ipsos,Congressional Job Approval,"Approve 25, Disapprove 63",Disapprove +38,2018,2018


In [31]:
today = polls_copy
senate = pd.DataFrame(columns=['Race/Topic (Click to Sort)','Poll','Results','Spread', 'Date'])
for element in range(len(today)):
    race = today.iloc[element,:]["Race/Topic (Click to Sort)"]
    z = re.match(r".*[Ss]enate.*", race)
    if z:
        senate = senate.append(today.iloc[element,:], ignore_index = True)
senate.head()

,Race/Topic (Click to Sort),Poll,Results,Spread,Date,Unnamed: 0,Year,year
0,Florida Senate - Rubio vs. Meek vs. Crist,Quinnipiac,"Crist 32, Rubio 30, Meek 24",Crist +2,2010-04-15,18998.0,2010.0,2010.0
1,Arkansas Senate - Boozman vs. Lincoln,R2000/Daily Kos (D),"Boozman 50, Lincoln 43",Boozman +7,2010-04-15,18992.0,2010.0,2010.0
2,Arkansas Senate - Boozman vs. Halter,R2000/Daily Kos (D),"Boozman 48, Halter 41",Boozman +7,2010-04-15,18993.0,2010.0,2010.0
3,Florida Senate - Rubio vs. Meek,Quinnipiac,"Rubio 42, Meek 38",Rubio +4,2010-04-15,18997.0,2010.0,2010.0
4,Florida Senate - Republican Primary,Quinnipiac,"Rubio 56, Crist 33",Rubio +23,2010-04-15,18995.0,2010.0,2010.0


In [257]:
senate_2018 = senate[senate['year'] == 2018]
senate_2018_names = senate_2018['Results'].values.tolist()
flatten = lambda lst: [item for sublist in lst for item in sublist]  
senate_cands_in_progress = flatten([j.split(',') for j in [re.sub(r'[0-9]', "", i) for i in senate_2018_names]])
names = [i.strip() for i in senate_cands_in_progress]
names = [i.lower() for i in names]
names

['mcsally',
 'arpaio',
 'ward',
 'stabenow',
 'james',
 'stabenow',
 'pensler',
 'nelson',
 'scott',
 'feinstein',
 'de leon',
 'nelson',
 'scott',
 'cruz',
 'jacobson',
 'miller',
 'sam',
 'de stefano',
 "o'rourke",
 'hernandez',
 'kimbrough',
 'wicker',
 'mcdaniel',
 'romney',
 'wilson',
 'nelson',
 'scott',
 'heitkamp',
 'cramer',
 'nelson',
 'scott',
 'hawley',
 'mccaskill',
 'menendez',
 'hugin',
 'gillibrand',
 'farley',
 'nelson',
 'scott',
 'feinstein',
 'schrader',
 'de leon',
 'kalemkarian',
 'little',
 'melendez',
 'casey',
 'barletta',
 'bredesen',
 'blackburn',
 'menendez',
 'hugin',
 'mccaskill',
 'hawley',
 'casey',
 'christiana',
 'casey',
 'barletta',
 'mcsally',
 'ward',
 'arpaio',
 'gillibrand',
 'farley',
 'cruz',
 "o'rourke",
 'braun',
 'rokita',
 'messer',
 'sinema',
 'mcsally',
 'sinema',
 'ward',
 'mcsally',
 'ward',
 'arpaio',
 'sinema',
 'arpaio',
 'mccaskill',
 'hawley',
 'morrisey',
 'jenkins',
 'blankenship',
 'heller',
 'rosen',
 'bredesen',
 'blackburn',


In [258]:
# polls_copy